In [1]:
# run this after
#     ./1_populate_tables.ipynb
#     ./2_normalize_features_by_speaker.ipynb

In [2]:
import pandas as pd

import sys
sys.path.append('../python/')
from measures import gCon, gSim, lCon, lSim, syn, getSesIDs
from analysis import bh, spToStr, printRes, ttest_ind, ttest_rel, \
    compareGP, testDurCorr, testSigF, testSigGP, two_factor_analysis
from constants import FEATURES

ses_ids = getSesIDs()

## Significance per feature and per feature and session

Paired t-test of measure vs. baseline for each feature (and per session for local measures)

In [3]:
# local similarity
alpha = 0.05
df = testSigF(lSim, alpha)

# t-test per session and feature
tp = {}
for ses_id in ses_ids:
    tp[ses_id] = []
    for f in FEATURES:
        f1 = df['f'] == f
        f2 = df['tsk_ses_id'] == ses_id
        tp[ses_id] += [ttest_rel(df[f1&f2]['main'].values, 
                                 df[f1&f2]['baseline'].values)]
# significance threshold per session
p_sigs = {ses_id: bh([v[1] for v in tp[ses_id]], alpha) 
          for ses_id in ses_ids}
p_apps = {ses_id: bh([v[1] for v in tp[ses_id]], 2*alpha) 
          for ses_id in ses_ids}
# results
data = [[spToStr(tp[ses_id][j][0], tp[ses_id][j][1], 
                 p_sigs[ses_id], p_apps[ses_id]) 
         for ses_id in ses_ids] 
       for j, f in enumerate(FEATURES)]
pd.DataFrame(data, FEATURES, ses_ids)

intensity_mean 3.861952943358367e-06 +4.624088027936923 4770
intensity_max 0.0003173868570504122 -3.6033164229663783 4770
pitch_mean (0.5000202477338598)   4752
pitch_max (0.0650468005891044)   4752
jitter (0.08836835293583103)   4517
shimmer (0.14302111485075786)   4501
nhr (0.31925271677204436)   4759
rate_syl (0.8040172297416475)   4770



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
intensity_mean,,,,,,,,,,,(+),,,,,+
intensity_max,,,,,,,,,,,-,,,,-,
pitch_mean,,,,,,,,,,,+,,,,,
pitch_max,,,,,,,,,,,,,(-),,,
jitter,,,,,,,,,,,,,,,,
shimmer,,,,,,,,,,,,,,,+,
nhr,,,,,,,,,,,,,,,,
rate_syl,,,,,,,,,,,,,,,,


In [4]:
# synchrony
alpha = 0.05
df = testSigF(syn, alpha)

p_sigs = [bh(df[df['tsk_ses_id'] == ses_id]['p_real'], alpha) 
          for ses_id in ses_ids]
func = lambda r: p_sigs[r.tsk_ses_id]
df['p_sig'] = df.apply(func, axis=1)
p_apps = [bh(df[df['tsk_ses_id'] == ses_id]['p_real'], 2*alpha) 
          for ses_id in ses_ids]
func = lambda r: p_apps[r.tsk_ses_id]
df['p_app'] = df.apply(func, axis=1)
func = lambda r: r.p_real <= p_sigs[r.tsk_ses_id] and r.sig_fake < 2
df['sig'] = df.apply(func, axis=1)
func = lambda r: r.p_real > p_sigs[r.tsk_ses_id] \
             and r.p_real <= p_apps[r.tsk_ses_id] \
             and r.sig_fake < 2
df['app'] = df.apply(func, axis=1)
func = lambda r: ('(' if r.app else '') \
                + (('+' if r.main > 0 else '-') if r.sig or r.app \
                   else ' ') \
                + (')' if r.app else '')
df['out'] = df.apply(func, axis=1)

data = [[df[(df['tsk_ses_id'] == ses_id)&
            (df['f'] == f)]['out'].values[0] 
         for ses_id in ses_ids]
        for f in FEATURES]
pd.DataFrame(data, FEATURES, ses_ids)

intensity_mean 3.677429238716533e-05 +5.773180870402556 15
intensity_max 0.000501153351525589 -4.415461456117255 15
pitch_mean (0.24052559411950733)   15
pitch_max 0.001620078633922638 -3.835849164162116 15
jitter (0.8956502885473334)   15
shimmer (0.7215794856995197)   15
nhr 0.00020232454327080867 +4.874079217197889 15
rate_syl (0.9339609419985779)   15



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
intensity_mean,,+,,,,,(+),(+),+,,+,,,,(+),+
intensity_max,,-,,,,,,-,,,-,-,,-,,
pitch_mean,,,,,,,,,,,+,,,,,
pitch_max,,,,-,,,,(-),,,,,,,,
jitter,,,,,,,,(+),,,,,,,,
shimmer,,,,,,,,,,,,,,,+,
nhr,,,,,,+,,,,,,,,,+,
rate_syl,,,,,,,,,,(-),,,,-,(+),


In [5]:
# convergence (seed set to ensure stable results)
alpha = 0.05
df = testSigF(lCon, alpha)

p_sigs = [bh(df[df['tsk_ses_id'] == ses_id]['p_real'], alpha) 
          for ses_id in ses_ids]
func = lambda r: p_sigs[r.tsk_ses_id]
df['p_sig'] = df.apply(func, axis=1)
p_apps = [bh(df[df['tsk_ses_id'] == ses_id]['p_real'], 2*alpha) 
          for ses_id in ses_ids]
func = lambda r: p_apps[r.tsk_ses_id]
df['p_app'] = df.apply(func, axis=1)
func = lambda r: r.p_real <= p_sigs[r.tsk_ses_id] and r.sig_fake < 2
df['sig'] = df.apply(func, axis=1)
func = lambda r: r.p_real > p_sigs[r.tsk_ses_id] \
             and r.p_real <= p_apps[r.tsk_ses_id] \
             and r.sig_fake < 2
df['app'] = df.apply(func, axis=1)
func = lambda r: ('(' if r.app else '') \
                + (('+' if r.main > 0 else '-') if r.sig or r.app \
                   else ' ') \
                + (')' if r.app else '')
df['out'] = df.apply(func, axis=1)

data = [[df[(df['tsk_ses_id'] == ses_id)&
            (df['f'] == f)]['out'].values[0] 
         for ses_id in ses_ids]
        for f in FEATURES]
pd.DataFrame(data, FEATURES, ses_ids)

intensity_mean (0.819994638709211)   15
intensity_max (0.0734223515921703)   15
pitch_mean (0.2540596860660116)   15
pitch_max (0.3090421010788501)   15
jitter (0.3842876256100971)   15
shimmer (0.159384178188147)   15
nhr (0.21338138502870357)   15
rate_syl (0.5067112625803454)   15



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
intensity_mean,+,,,,,,,,,,,-,,,,
intensity_max,+,,-,-,-,,+,(-),,-,(-),-,,,-,
pitch_mean,-,,,,,,,,,,,,,,,
pitch_max,-,,+,,,,,,(+),(+),,,,,,+
jitter,,,-,,,,+,,,,,,,,,+
shimmer,,,(-),,,,,,,,,,,,,+
nhr,,,-,,,,(+),,,,,,,,,
rate_syl,,,(-),,,,,,,,,,,,,


In [6]:
# global similarity (aggregating per task)
df = testSigF(gSim, kwargs={'tsk_or_ses': 'tsk'})

intensity_mean (0.061120635672380665)   31
intensity_max (0.7535594265976391)   31
pitch_mean (0.7637990173390787)   31
pitch_max (0.8118055285344494)   31
jitter (0.964765307015344)   31
shimmer (0.14598848657319646)   31
nhr (0.1619700311606446)   31
rate_syl (0.31482227162663384)   31



In [7]:
# global convergence
df = testSigF(gCon, kwargs={'tsk_or_ses': 'tsk'})

intensity_mean (0.08383008495000566)   31
intensity_max (0.008831586590106303) (-2.79474745752767) 31
pitch_mean (0.1173247369570391)   31
pitch_max (0.016831399275534578) (-2.5267420570067403) 31
jitter (0.23185648878065462)   31
shimmer (0.7984441078292082)   31
nhr (0.5725650161727789)   31
rate_syl (0.7741563310749484)   31



## Significance per gender pair

In [8]:
testSigGP(lSim, kwargs={'tsk_or_ses': 'tsk'})

,f,m,fm
intensity_mean,,+,
intensity_max,,,
pitch_mean,,,
pitch_max,,,
jitter,,,
shimmer,,,
nhr,,,
rate_syl,,,


In [9]:
# local similarity
testSigGP(lSim)

,f,m,fm
intensity_mean,,+,
intensity_max,,,
pitch_mean,,,
pitch_max,,,
jitter,,,
shimmer,,,
nhr,,,
rate_syl,,,


In [10]:
testSigGP(syn, kwargs={'tsk_or_ses': 'tsk'})

,f,m,fm
intensity_mean,+,,
intensity_max,-,,-
pitch_mean,,,
pitch_max,-,,
jitter,,,
shimmer,,,
nhr,+,,
rate_syl,,,


In [11]:
# synchrony
testSigGP(syn)

,f,m,fm
intensity_mean,+,(+),
intensity_max,(-),,-
pitch_mean,,,
pitch_max,-,,(-)
jitter,,,
shimmer,,,
nhr,+,,+
rate_syl,,,


In [12]:
testSigGP(lCon, kwargs={'tsk_or_ses': 'tsk'})

,f,m,fm
intensity_mean,,,
intensity_max,(-),,(-)
pitch_mean,,,
pitch_max,,,
jitter,,,
shimmer,,,
nhr,,,
rate_syl,,,


In [13]:
# convergence
testSigGP(lCon)

,f,m,fm
intensity_mean,,,
intensity_max,,,-
pitch_mean,,,
pitch_max,,,
jitter,,,
shimmer,,,
nhr,,,
rate_syl,,,


In [14]:
# global similarity
testSigGP(gSim, kwargs={'tsk_or_ses': 'tsk'})

,f,m,fm
intensity_mean,,,
intensity_max,,,
pitch_mean,,,
pitch_max,,,
jitter,,,
shimmer,,,
nhr,,,
rate_syl,,,


In [15]:
# global convergence
testSigGP(gCon, kwargs={'tsk_or_ses': 'tsk'})

,f,m,fm
intensity_mean,,,
intensity_max,,,
pitch_mean,,,
pitch_max,,,
jitter,,,
shimmer,,,
nhr,,,
rate_syl,,,


## Gender pair differences (symmetric measures)

In [16]:
# local similarity
compareGP(lSim)

,ff vs. mm,ff vs. fm,mm vs. fm
intensity_mean,,,
intensity_max,,,
pitch_mean,,+,+
pitch_max,-,-,(+)
jitter,,,
shimmer,,,
nhr,,(+),
rate_syl,,,


In [17]:
# synchrony
compareGP(syn)

,ff vs. mm,ff vs. fm,mm vs. fm
intensity_mean,,,
intensity_max,,,
pitch_mean,,,
pitch_max,(-),,
jitter,,,
shimmer,,,
nhr,+,+,
rate_syl,,,


In [18]:
# convergence
compareGP(lCon)

,ff vs. mm,ff vs. fm,mm vs. fm
intensity_mean,,,
intensity_max,,(+),(+)
pitch_mean,,,
pitch_max,,,
jitter,,(+),(+)
shimmer,,,
nhr,,,
rate_syl,,,


In [19]:
# global similarity (aggregating by task)
compareGP(gSim, kwargs={'tsk_or_ses': 'tsk'})

,ff vs. mm,ff vs. fm,mm vs. fm
intensity_mean,,,
intensity_max,,,
pitch_mean,,,
pitch_max,-,,
jitter,,,
shimmer,,,
nhr,,,
rate_syl,,,


In [20]:
# global convergence
compareGP(gCon, kwargs={'tsk_or_ses': 'tsk'})

,ff vs. mm,ff vs. fm,mm vs. fm
intensity_mean,,,
intensity_max,,,
pitch_mean,,,
pitch_max,,,
jitter,,,
shimmer,,,
nhr,,,
rate_syl,,,


## Gender differences (asymmetric local measures)

comparison of trend/strength of male and female respondents, per measure and feature ('+' means males entrain more)

In [21]:
# local similarity
res = [(f,) + ttest_ind(lSim(f, g2='m')['main'], 
                        lSim(f, g2='f')['main'])
       for f in FEATURES]
printRes(res)

intensity_mean (0.1698028448631103)   4769
intensity_max (0.30518659106675206)   4769
pitch_mean (0.11219107600073414)   4751
pitch_max (0.013672724016461017)   4751
jitter (0.44119241741223403)   4516
shimmer (0.3622778051654537)   4500
nhr (0.4914635540978647)   4758
rate_syl (0.7093238370679384)   4769



In [22]:
# synchrony (aggregated per task)
res = [(f,) + ttest_ind(syn(f, 'tsk', g2='m')['main'], 
                        syn(f, 'tsk', g2='f')['main'])
       for f in FEATURES]
printRes(res)

intensity_mean (0.3627031698027865)   62
intensity_max (0.5699740916671695)   62
pitch_mean (0.2925268851915608)   62
pitch_max (0.22956137480444155)   62
jitter (0.9144585152979359)   62
shimmer (0.2243960735697352)   62
nhr (0.6276646295415602)   62
rate_syl (0.36802556164381306)   62



In [23]:
# synchrony (aggregated per session)
res = [(f,) + ttest_ind(syn(f, g2='m')['main'], 
                        syn(f, g2='f')['main'])
       for f in FEATURES]
printRes(res)

intensity_mean (0.31422103621791575)   30
intensity_max (0.9185565903218537)   30
pitch_mean (0.7240391535558252)   30
pitch_max (0.07579019157880036)   30
jitter (0.49059906508555773)   30
shimmer (0.20519858154014853)   30
nhr (0.1991493995892898)   30
rate_syl (0.33414979050842253)   30



In [24]:
# convergence (aggregated per task)
res = [(f,) + ttest_ind(lCon(f, 'tsk', g2='m')['main'], 
                        lCon(f, 'tsk', g2='f')['main'])
       for f in FEATURES]
printRes(res)

intensity_mean (0.14203280507150112)   62
intensity_max (0.8274725388336769)   62
pitch_mean (0.529081709145539)   62
pitch_max (0.0521709182043457)   62
jitter (0.2095851281895581)   62
shimmer (0.9848508275938778)   62
nhr (0.6146497821712718)   62
rate_syl (0.3436561136386368)   62



In [25]:
# convergence (aggregated per session)
res = [(f,) + ttest_ind(lCon(f, g2='m')['main'], 
                        lCon(f, g2='f')['main'])
       for f in FEATURES]
printRes(res)

intensity_mean (0.7273074239065018)   30
intensity_max (0.9903229476849236)   30
pitch_mean (0.5839181316551887)   30
pitch_max (0.6624302833133772)   30
jitter (0.5665132759296492)   30
shimmer (0.6253336056009764)   30
nhr (0.4039760952249791)   30
rate_syl (0.9934178168684542)   30



## Role differences (asymmetric local measures)

comparison of trend/strength of describer and follower responses, per measure and feature ('+' means describers entrain more)

In [26]:
# local similarity
res = [(f,) + ttest_ind(lSim(f, role='d')['main'], 
                        lSim(f, role='f')['main'])
       for f in FEATURES]
printRes(res)

intensity_mean (0.04539156299226481) (-2.0015327140499335) 4769
intensity_max (0.17051245149695357)   4769
pitch_mean (0.16486859982612295)   4751
pitch_max (0.6036221940064848)   4751
jitter (0.014927228838533767) (-2.4350732874595864) 4516
shimmer (0.023564163436815885) (-2.264932703131463) 4500
nhr 0.005625289581044994 -2.770120839965428 4758
rate_syl (0.08941112246129251)   4769



In [30]:
from numpy import mean
f = 'nhr'
print(f, 'd', mean(lSim(f, role='d')['main']))
print(f, 'f', mean(lSim(f, role='f')['main']))
f = 'jitter'
print(f, 'd', mean(lSim(f, role='d')['main']))
print(f, 'f', mean(lSim(f, role='f')['main']))
f = 'shimmer'
print(f, 'd', mean(lSim(f, role='d')['main']))
print(f, 'f', mean(lSim(f, role='f')['main']))
f = 'intensity_mean'
print(f, 'd', mean(lSim(f, role='d')['main']))
print(f, 'f', mean(lSim(f, role='f')['main']))


nhr d -0.9935800228309224
nhr f -0.9193923867306066
jitter d -0.9915594199549609
jitter f -0.9203122490101611
shimmer d -1.071788861100919
shimmer f -1.009028347246721
intensity_mean d -1.0547131900226971
intensity_mean f -1.0079915526857541


In [31]:
# synchrony (aggregating per task)
res = [(f,) + ttest_ind(syn(f, 'tsk', role='d')['main'], 
                        syn(f, 'tsk', role='f')['main'])
       for f in FEATURES]
printRes(res)

intensity_mean (0.7800679605826747)   62
intensity_max (0.8229147023506502)   62
pitch_mean (0.30756121634085176)   62
pitch_max (0.5481082311671429)   62
jitter (0.295788984368036)   62
shimmer (0.6826746769988508)   62
nhr (0.9918004161379921)   62
rate_syl (0.055522037496083816)   62



In [32]:
# convergence (aggregating per task)
res = [(f,) + ttest_ind(lCon(f, 'tsk', role='d')['main'], 
                        lCon(f, 'tsk', role='f')['main'])
       for f in FEATURES]
printRes(res)

intensity_mean (0.1028758323249617)   62
intensity_max (0.6194908764409857)   62
pitch_mean (0.9740090472361149)   62
pitch_max (0.9952483012948037)   62
jitter (0.23716604683458825)   62
shimmer (0.08193515249758945)   62
nhr (0.8637738508558013)   62
rate_syl (0.5580416912117352)   62



# Gender pair and role differences (asymmetric local measures)

In [33]:
data_all = two_factor_analysis(lSim, do_g2=True, do_role=True)

intensity_mean (sample size: 1096)
               ss  df       F     PR(>F)               f
g2:role  1.998407   1  3.0985  0.0784341  intensity_mean


intensity_max (sample size: 1096)
               ss  df         F    PR(>F)              f
g2:role  0.442052   1  0.636501  0.425024  intensity_max


pitch_mean (sample size: 1092)
               ss  df         F    PR(>F)           f
g2:role  0.658688   1  0.682919  0.408629  pitch_mean


pitch_max (sample size: 1092)
               ss  df         F    PR(>F)          f
g2:role  0.505805   1  0.564736  0.452399  pitch_max


jitter (sample size: 1046)
               ss  df         F    PR(>F)       f
g2:role  0.451897   1  0.452047  0.501402  jitter


shimmer (sample size: 1039)
               ss  df          F    PR(>F)        f
g2:role  0.011188   1  0.0129147  0.909526  shimmer


nhr (sample size: 1093)
               ss  df        F      PR(>F)    f
g2:role  5.913584   1  6.99763  0.00819101  nhr


rate_syl (sample size: 1096)
      

In [34]:
from numpy import mean
df = data_all['nhr']
print(mean(df[(df['g2'] == 'm')&(df['role'] == 'f')]['v']))
print(mean(df[(df['g2'] == 'm')&(df['role'] == 'd')]['v']))

-0.8927075418326738
-1.0370348311633943


In [35]:
data_all = two_factor_analysis(syn, do_g2=True, do_role=True)

intensity_mean (sample size: 14)
               ss  df          F   PR(>F)               f
g2:role  0.001049   1  0.0564509  0.81313  intensity_mean


intensity_max (sample size: 14)
               ss  df          F    PR(>F)              f
g2:role  0.000501   1  0.0277243  0.868404  intensity_max


pitch_mean (sample size: 14)
               ss  df         F    PR(>F)           f
g2:role  0.010732   1  0.679823  0.413414  pitch_mean


pitch_max (sample size: 14)
               ss  df        F    PR(>F)          f
g2:role  0.028305   1  1.30743  0.258097  pitch_max


jitter (sample size: 14)
               ss  df         F   PR(>F)       f
g2:role  0.006504   1  0.435967  0.51199  jitter


shimmer (sample size: 14)
              ss  df        F   PR(>F)        f
g2:role  0.03511   1  1.47844  0.22951  shimmer


nhr (sample size: 14)
               ss  df          F   PR(>F)    f
g2:role  0.001343   1  0.0786632  0.78023  nhr


rate_syl (sample size: 14)
              ss  df         F  

In [36]:
data_all = two_factor_analysis(lCon, do_g2=True, do_role=True)

intensity_mean (sample size: 14)
              ss  df        F    PR(>F)               f
g2:role  0.05554   1  2.43319  0.124857  intensity_mean


intensity_max (sample size: 14)
               ss  df         F    PR(>F)              f
g2:role  0.019393   1  0.599803  0.442161  intensity_max


pitch_mean (sample size: 14)
              ss  df         F    PR(>F)           f
g2:role  0.01783   1  0.583285  0.448479  pitch_mean


pitch_max (sample size: 14)
               ss  df         F    PR(>F)          f
g2:role  0.010673   1  0.336293  0.564481  pitch_max


jitter (sample size: 14)
             ss  df        F     PR(>F)       f
g2:role  0.1346   1  4.94748  0.0304947  jitter


shimmer (sample size: 14)
            ss  df        F    PR(>F)        f
g2:role  0.029   1  1.48569  0.228388  shimmer


nhr (sample size: 14)
               ss  df        F    PR(>F)    f
g2:role  0.026861   1  1.24121  0.270362  nhr


rate_syl (sample size: 14)
               ss  df        F      PR(>F)  

In [37]:
from numpy import mean
df = data_all['rate_syl']
print(mean(df[(df['g2'] == 'f')&(df['role'] == 'd')]['v']))
print(mean(df[(df['g2'] == 'm')&(df['role'] == 'd')]['v']))

-0.11725240863661039
0.018081841291949164


In [38]:
#two_factor_analysis(lSim, 'tsk', do_g1=True, do_g2=True)

In [39]:
#two_factor_analysis(lSim, 'tsk', do_g1=True, do_g2=True, do_role=True)

In [40]:
#two_factor_analysis(syn, 'tsk', do_g1=True, do_g2=True)

In [41]:
#two_factor_analysis(syn, 'tsk', do_g1=True, do_g2=True, do_role=True)

In [42]:
#two_factor_analysis(lCon, 'tsk', do_g1=True, do_g2=True)

In [43]:
#two_factor_analysis(lCon, 'tsk', do_g1=True, do_g2=True, do_role=True)

In [44]:
#two_factor_analysis(gSim, 'tsk', do_g1=True, do_g2=True)

In [45]:
#two_factor_analysis(gCon, 'tsk', do_g1=True, do_g2=True)

## Correlation between interaction length and measures

Pearson correlation between the duration of each task/session and entrainment, for each measure and each feature

In [46]:
df = testDurCorr(lSim, kwargs={'tsk_or_ses': 'ses'})

intensity_mean (0.8515554624043327)   14
intensity_max (0.9112615718632997)   14
pitch_mean (0.10469286778853079)   14
pitch_max (0.36627089266808704)   14
jitter (0.9236031244287888)   14
shimmer (0.5371489742055946)   14
nhr (0.3343291270498963)   14
rate_syl (0.15832480625553835)   14



In [47]:
df = testDurCorr(lSim, kwargs={'tsk_or_ses': 'tsk'})

intensity_mean (0.7752334459099379)   30
intensity_max (0.4873107101914921)   30
pitch_mean (0.8007450701275518)   30
pitch_max (0.5347790130648138)   30
jitter (0.9893953210142687)   30
shimmer (0.1462580071336205)   30
nhr (0.4565493351899992)   30
rate_syl (0.09536670985338544)   30



In [48]:
df = testDurCorr(syn, kwargs={'tsk_or_ses': 'ses'})

intensity_mean (0.866033648203915)   14
intensity_max (0.45240563484376484)   14
pitch_mean (0.6827559122092639)   14
pitch_max (0.9232535468190094)   14
jitter (0.3868501572645077)   14
shimmer (0.49407545303563716)   14
nhr (0.6525559544251223)   14
rate_syl (0.9674545430238999)   14



In [49]:
df = testDurCorr(syn, kwargs={'tsk_or_ses': 'tsk'})

intensity_mean (0.6057617283964379)   30
intensity_max (0.027650827814058115)   30
pitch_mean (0.6068373644090297)   30
pitch_max (0.6254563758353429)   30
jitter (0.06671566119208439)   30
shimmer (0.115767897311603)   30
nhr (0.2241183929843274)   30
rate_syl (0.7389473945409732)   30



In [50]:
df = testDurCorr(lCon, kwargs={'tsk_or_ses': 'ses'})

intensity_mean (0.2201376323397323)   14
intensity_max (0.8922048613823339)   14
pitch_mean (0.11617238937136286)   14
pitch_max (0.805030164633937)   14
jitter (0.7196954913101786)   14
shimmer (0.17098240161432304)   14
nhr (0.32963515803475224)   14
rate_syl (0.6583735806627483)   14



In [51]:
df = testDurCorr(lCon, kwargs={'tsk_or_ses': 'tsk'})

intensity_mean (0.30359594500918424)   30
intensity_max (0.8180600160697824)   30
pitch_mean (0.2949101118303764)   30
pitch_max (0.5800145631457481)   30
jitter (0.3308739865739744)   30
shimmer (0.027368718687001026)   30
nhr (0.8784242914212347)   30
rate_syl (0.17369517355305564)   30



In [52]:
df = testDurCorr(gSim, kwargs={'tsk_or_ses': 'tsk'})

intensity_mean (0.29752274994879324)   30
intensity_max 0.0023459149640038105 +0.5188441274210832 30
pitch_mean (0.04672907077537342) (+0.35416620535199567) 30
pitch_max (0.03169179333077265) (+0.380488781198179) 30
jitter 0.005488841762943875 +0.47949313331535987 30
shimmer (0.27009034190544096)   30
nhr (0.9383989399711227)   30
rate_syl (0.04340701539878168) (+0.35932006651469944) 30



In [53]:
df = testDurCorr(gCon, kwargs={'tsk_or_ses': 'tsk', 'dod': True})

intensity_mean (0.7091330139988163)   30
intensity_max (0.8304229827974732)   30
pitch_mean (0.2884240569478503)   30
pitch_max (0.9891922538192344)   30
jitter (0.0820461668520978)   30
shimmer (0.14364828816393885)   30
nhr (0.8344414650510856)   30
rate_syl (0.8395376558377935)   30

